In [ ]:
import copy
import pygame
import random
import math
import time

In [ ]:
# -------------------------------------------------------------------------
# Konstante
# -------------------------------------------------------------------------

# Dimenzije igre
FOOTER_HEIGHT = 60
WIDTH, HEIGHT = 900, 900
ROWS, COLS = 8, 8
BORDER_WIDTH = 18

# Boje ploce
WHITE = (227, 227, 227)
BLACK = (30, 30, 30)
BORDER = (13, 11, 9)

# Boje igraca
PLAYER1 = (156, 181, 170) 
PLAYER2 = (0, 0, 0)        

# Border
BOARD_WIDTH = WIDTH - 2 * BORDER_WIDTH
BOARD_HEIGHT = HEIGHT - 2 * BORDER_WIDTH
SQUARE_SIZE = BOARD_WIDTH // COLS

# -------------------------------------------------------------------------
# Klase
# -------------------------------------------------------------------------

# Figure, logika i prikaz
class Figura:
    RUB = 2
    PADDING = 5
    
    def __init__(self, row, col, color):
        self.row = row
        self.col = col
        self.color = color
        self.king = False  # Default
        # Smjer u kojem igra svaki igrac
        if self.color == PLAYER1:
            self.direction = 1
        else:
            self.direction = -1
        self.x = 0
        self.y = 0

    def promote(self):
        self.king = True  # Promocija u kralja

    def polozaj(self, screen):
        # Centar polja
        self.x = self.col * SQUARE_SIZE + SQUARE_SIZE // 2 + BORDER_WIDTH
        self.y = self.row * SQUARE_SIZE + SQUARE_SIZE // 2 + BORDER_WIDTH

        # Radijus
        radijus = SQUARE_SIZE // 2.5 - self.PADDING

        # Prikaz figure
        pygame.draw.circle(screen, (68, 68, 68), (self.x, self.y), radijus + self.RUB)
        pygame.draw.circle(screen, self.color, (self.x, self.y), radijus)

        # Indikator za kralja
        if self.king:
            pygame.draw.circle(screen, (255, 223, 0), (self.x, self.y), radijus // 2)  # "kruna"

# Glavna logika igre i pravila
class Game:
    def __init__(self):
        self.status = 'playing'
        
        # Pracenje poteza: 0 za P1, a 1 za P2
        self.turn = 0  
        
        # Reprezentativne boje igraca
        self.players = [PLAYER1, PLAYER2]
        
        # Brojac za broj kraljeva/obicnih figura na ploci
        self.tokens = [12, 12] 
        self.kings = [0, 0]   
        
        # Trenutno odabrana figura
        self.selected_piece = None  
        self.jumping = False

        # Polja koja ce se posvjetliti
        self.highlighted_squares = []

        # Prikaz na prozoru o igracu na potezu
        player_name = "Player1" if self.turn == 0 else "Player2"
        pygame.display.set_caption(f"{player_name}'s turn")

        # Pocetni skup
        self.pieces = []
        self.init_pieces()

    # Inicijalizacija figura
    def init_pieces(self):
        self.pieces.clear()
        
        # Postavljanje figura na crna polja
        for row in range(ROWS):
            for col in range(COLS):
                # Za svako crno polje
                if (row + col) % 2 != 0:
                    # P1 figure na prva 3 reda
                    if row < 3:
                        self.pieces.append(Figura(row, col, PLAYER1))
                    # P2 figure na zadnja 3 reda
                    elif row > 4:
                        self.pieces.append(Figura(row, col, PLAYER2))

    # Prikaz ploce
    def draw_board(self, screen):
        # Border
        pygame.draw.rect(screen, BORDER, (0, 0, WIDTH, HEIGHT), BORDER_WIDTH)

        # Polja
        for row in range(ROWS):
            for col in range(COLS):
                x = BORDER_WIDTH + col * SQUARE_SIZE
                y = BORDER_WIDTH + row * SQUARE_SIZE
                color = WHITE if (row + col) % 2 == 0 else BLACK
                pygame.draw.rect(screen, color, (x, y, SQUARE_SIZE, SQUARE_SIZE))

    def draw(self, screen):
        # Prvo se crta polje
        self.draw_board(screen)
        # Dodaju se figure
        for piece in self.pieces:
            piece.polozaj(screen)
        # Bojanje mogucih poteza
        self.draw_highlighted_squares(screen)

    def draw_highlighted_squares(self, screen):
        highlight_color = (255, 255, 0, 80) 
        highlight_surface = pygame.Surface((SQUARE_SIZE, SQUARE_SIZE), pygame.SRCALPHA)
        highlight_surface.fill(highlight_color)

        for (row, col) in self.highlighted_squares:
            x = BORDER_WIDTH + col * SQUARE_SIZE
            y = BORDER_WIDTH + row * SQUARE_SIZE
            screen.blit(highlight_surface, (x, y))

    # Funkcije da se figure prebroje i lociraju
    def piece_at(self, row, col):
        # Ako figura postoji, vraca se pozicija (row, col), ako ne vraca se None
        for p in self.pieces:
            if p.row == row and p.col == col:
                return p
        return None

    def get_pieces_of_color(self, color):
        # Lista svih figura pojedinog igraca
        return [p for p in self.pieces if p.color == color]

    def count_pieces(self, color):
        # Figure trenutno preostale na ploci
        return len(self.get_pieces_of_color(color))

    def count_kings(self, color):
        # Broj kraljeva na ploci
        return sum(p.king for p in self.get_pieces_of_color(color))

    # Logika iza kretanja i pravila
    def evaluate_click(self, mouse_pos):
        
        # Mora se odabrati crna figura, tj figura igraca P2
        # Moze se pokusati pomaknuti odabrana figura, ukoliko postoje validni potezi
        # Provjera kraja igre, ako je igra gotova mora se ponovno kliknuti 
        
        if self.status == 'playing':
            row = self.get_clicked_row(mouse_pos)
            col = self.get_clicked_col(mouse_pos)
            current_color = self.players[self.turn % 2]

            if self.selected_piece:
                # Ako je odabrana figura, provjera poteza
                valid, jumped_piece = self.is_valid_move(self.selected_piece, (row, col))

                if valid:
                    yes = self.play(self.selected_piece, (row, col), jumped_piece)
                    # Micanje boje mogucih poteza nakon sto je napravljen potez
                    self.highlighted_squares = []
                else:
                    # Ako se klikne nazad na isto polje
                    if (row == self.selected_piece.row and col == self.selected_piece.col):
                        self.selected_piece = None
                        self.highlighted_squares = []
                        if self.jumping:
                            self.jumping = False
                            self.next_turn()
                    else:
                        print("Invalid move.")
            else:
                # Odabrana figura
                maybe_piece = self.piece_at(row, col)
                if maybe_piece and maybe_piece.color == current_color:
                    self.selected_piece = maybe_piece
                    # Boja za moguce poteze
                    self.highlighted_squares = []
                    moves = self.get_all_moves_for_piece(maybe_piece)
                    for (_, (r, c), _) in moves:
                        self.highlighted_squares.append((r, c))
                else:
                    self.selected_piece = None
                    self.highlighted_squares = []
        else:
            # Reset klik za kraj igre
            self.__init__()

    def is_valid_move(self, piece, to_loc):
        # Provjera 
        from_row, from_col = piece.row, piece.col
        to_row, to_col = to_loc

        # Ako je polje zauzeto
        if self.piece_at(to_row, to_col) is not None:
            return (False, None)

        row_diff = to_row - from_row
        col_diff = abs(to_col - from_col)

        # ---------- Obicna figura --------------
        if not piece.king:
            # Obican potez
            if row_diff == piece.direction and col_diff == 1 and not self.jumping:
                # Bez skakanja
                return (True, None)

            # Ako se napada
            if row_diff == 2 * piece.direction and col_diff == 2:
                mid_row = (from_row + to_row) // 2
                mid_col = (from_col + to_col) // 2
                jumped = self.piece_at(mid_row, mid_col)
                # Mora papati protivnika
                if jumped and jumped.color != piece.color:
                    return (True, jumped)

        # ---------- Kralj --------------
        else:
            # Kretanje za jedno polje u bilo kojem smjeru
            if abs(row_diff) == 1 and col_diff == 1 and not self.jumping:
                return (True, None)

            # Moze pojesgt figure ispred i iza sebe
            if abs(row_diff) == 2 and col_diff == 2:
                mid_row = (from_row + to_row) // 2
                mid_col = (from_col + to_col) // 2
                jumped = self.piece_at(mid_row, mid_col)
                if jumped and jumped.color != piece.color:
                    return (True, jumped)

        return (False, None)

    def get_valid_moves(self, color):
        # Dohvat svih valid poteza
        moves = []
        pieces = self.get_pieces_of_color(color)
        for piece in pieces:
            piece_moves = self.get_all_moves_for_piece(piece)
            moves.extend(piece_moves)
        return moves

    def get_all_moves_for_piece(self, piece):
        # Valid potezi za jednu figuru
        results = []
        if not piece.king:
            # Non-king => jedan korak za +/- 1 redak
            row_single = piece.row + piece.direction
            col_left = piece.col - 1
            col_right = piece.col + 1
            for c in (col_left, col_right):
                if 0 <= row_single < ROWS and 0 <= c < COLS:
                    valid, jumped = self.is_valid_move(piece, (row_single, c))
                    if valid:
                        results.append((piece, (row_single, c), jumped))

            # Jump => korak za +/- 2 reda
            row_jump = piece.row + 2 * piece.direction
            col_left_jump = piece.col - 2
            col_right_jump = piece.col + 2
            for c in (col_left_jump, col_right_jump):
                if 0 <= row_jump < ROWS and 0 <= c < COLS:
                    valid, jumped = self.is_valid_move(piece, (row_jump, c))
                    if valid:
                        results.append((piece, (row_jump, c), jumped))

        else:
            # King => +/- 1 redak za normalan korak u sivm smjerovima, 2 za napad
            # Obicno
            for dr in (+1, -1):
                for dc in (+1, -1):
                    r = piece.row + dr
                    c = piece.col + dc
                    if 0 <= r < ROWS and 0 <= c < COLS:
                        valid, jumped = self.is_valid_move(piece, (r, c))
                        if valid:
                            results.append((piece, (r, c), jumped))
            # Skakanje
            for dr in (+2, -2):
                for dc in (+2, -2):
                    r = piece.row + dr
                    c = piece.col + dc
                    if 0 <= r < ROWS and 0 <= c < COLS:
                        valid, jumped = self.is_valid_move(piece, (r, c))
                        if valid:
                            results.append((piece, (r, c), jumped))

        return results

    def play(self, piece, to_loc, jumped_piece, auto=False):
        to_row, to_col = to_loc
        from_row, from_col = piece.row, piece.col

        # Pomkani figuru
        piece.row = to_row
        piece.col = to_col

        # Ako napada
        if jumped_piece:
            self.pieces.remove(jumped_piece)
            jumped_index = 0 if jumped_piece.color == PLAYER1 else 1
            self.tokens[jumped_index] -= 1

            if not auto:
                # Ostavi istu figuru selektanu ako zelimo nastaviti skakati/napadati
                self.selected_piece = piece
                self.jumping = True
        else:
            # Ako nema dodatnog napada => prekini jump sequence
            self.selected_piece = None
            self.jumping = False

        # Promocije
        if not piece.king:
            # Promocija ako figura P1 dodje do dna
            if piece.color == PLAYER1 and piece.row == ROWS - 1:
                piece.promote()
                self.kings[0] += 1
            # Promocija P2 ako dodje do vrha
            elif piece.color == PLAYER2 and piece.row == 0:
                piece.promote()
                self.kings[1] += 1

        # Kraj poteza ako nismo skakali ili je set na auto
        if auto or (jumped_piece is None):
            self.next_turn()

        winner = self.check_winner()
        if winner is not None:
            self.status = 'game over'
        return winner

    def next_turn(self):
        self.turn += 1
        player_name = "Player1" if self.turn % 2 == 0 else "Player2"
        pygame.display.set_caption(f"{player_name}'s turn")

    def check_winner(self):
        num_p1 = self.count_pieces(PLAYER1)
        num_p2 = self.count_pieces(PLAYER2)
        
        # Ako nema figura
        if num_p1 == 0:
            return "Player2"
        if num_p2 == 0:
            return "Player1"
            
        #Ako nema poteza
        current_color = self.players[self.turn % 2]
        if len(self.get_valid_moves(current_color)) == 0 and not self.jumping:
            if current_color == PLAYER1:
                return "Player2"
            else:
                return "Player1"

        # Optimalan slucaj za izjednacenje
        if num_p1 == 1 and num_p2 == 1:
            return 'draw'

        return None

    # Pomak misa
    def get_clicked_row(self, mouse_pos):
        y = mouse_pos[1]

        # Ako se klikne na footer, return -1
        if y >= HEIGHT:
            return -1

        for i in range(1, ROWS):
            if y < i * HEIGHT / ROWS:
                return i - 1
        return ROWS - 1


    def get_clicked_col(self, mouse_pos):
        x = mouse_pos[0]
        for i in range(1, COLS):
            if x < i * WIDTH / COLS:
                return i - 1
        return COLS - 1

    def reset_current_turn(self):
        self.selected_piece = None
        self.highlighted_squares = []
        self.jumping = False

# -------------------------------------------------------------------------
# EVALUACIJA I MINIMAX 
# -------------------------------------------------------------------------
def evaluate(game, max_player_index):
    p1_pieces = game.count_pieces(PLAYER1)
    p2_pieces = game.count_pieces(PLAYER2)
    p1_kings = game.count_kings(PLAYER1)
    p2_kings = game.count_kings(PLAYER2)

    if max_player_index == 0:  # computer is Player1
        return (p1_pieces - p2_pieces) + 0.5 * (p1_kings - p2_kings)
    else:  # Player2
        return (p2_pieces - p1_pieces) + 0.5 * (p2_kings - p1_kings)


def minimax(game, depth, max_player_index, alpha=float('-inf'), beta=float('inf')):
    current_player_index = game.turn % 2
    winner = game.check_winner()
    if winner is not None:  # game over
        if winner == "Player1":
            return (math.inf if max_player_index == 0 else -math.inf, None)
        elif winner == "Player2":
            return (math.inf if max_player_index == 1 else -math.inf, None)
        else:  # 'draw'
            return (0, None)

    if depth == 0:
        return (evaluate(game, max_player_index), None)

    valid_moves = game.get_valid_moves(game.players[current_player_index])

    if current_player_index == max_player_index:
        # Maximizing
        best_eval = float('-inf')
        best_move = None
        for (piece, to_loc, jumped_piece) in valid_moves:
            new_game = copy.deepcopy(game)
            # We must find the equivalent piece in the new_game
            new_piece = find_equivalent_piece(new_game, piece)
            new_jumped = find_equivalent_piece(new_game, jumped_piece)
            new_game.play(new_piece, to_loc, new_jumped, auto=True)

            eval_child, _ = minimax(new_game, depth - 1, max_player_index, alpha, beta)
            if eval_child > best_eval:
                best_eval = eval_child
                best_move = (piece, to_loc, jumped_piece)
            alpha = max(alpha, eval_child)
            if beta <= alpha:
                break
        return (best_eval, best_move)
    else:
        # Minimizing
        best_eval = float('inf')
        best_move = None
        for (piece, to_loc, jumped_piece) in valid_moves:
            new_game = copy.deepcopy(game)
            new_piece = find_equivalent_piece(new_game, piece)
            new_jumped = find_equivalent_piece(new_game, jumped_piece)
            new_game.play(new_piece, to_loc, new_jumped, auto=True)

            eval_child, _ = minimax(new_game, depth - 1, max_player_index, alpha, beta)
            if eval_child < best_eval:
                best_eval = eval_child
                best_move = (piece, to_loc, jumped_piece)
            beta = min(beta, eval_child)
            if beta <= alpha:
                break
        return (best_eval, best_move)


def find_equivalent_piece(new_game, old_piece):
    # Dohvat figure za kopiju igre
    if old_piece is None:
        return None
    for p in new_game.pieces:
        if (p.row == old_piece.row and
            p.col == old_piece.col and
            p.color == old_piece.color and
            p.king == old_piece.king):
            return p
    return None

def draw_reset_button(screen):
    # Info text
    info_font = pygame.font.SysFont("cambria", 22)
    info_text = "Klikni gumb ako ti ne dozvoljava da vise napravis ikakav potez."
    info_surface = info_font.render(info_text, True, (255, 255, 255)) 
    screen.blit(info_surface, (10, HEIGHT + 10))
    
    button_width = 120
    button_height = 40
    # Pozicija u gornje desnom kutu
    button_x = WIDTH - button_width - 20
    button_y = HEIGHT
    
    button_rect = pygame.Rect(button_x, button_y, button_width, button_height)
    button_color = WHITE

    # Pravokutnik
    pygame.draw.rect(screen, button_color, button_rect)

    # Labela
    font = pygame.font.SysFont("cambria", 22) 
    text_surface = font.render("Reset Turn", True, (0, 0, 0))
    text_rect = text_surface.get_rect(center=button_rect.center)
    screen.blit(text_surface, text_rect)

    return button_rect

# -------------------------------------------------------------------------
# MAIN LOOP
# -------------------------------------------------------------------------
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT + FOOTER_HEIGHT))
    sat = pygame.time.Clock()

    game = Game()
    done = False

    # False za igru u 2 ljudi
    run_minimax = True
    # DUbina za minimax
    depth = 5
    # AI? 0 => Player1, 1 => Player2
    max_player = 0

    while not done:
        # AI potez => run minimax
        if game.turn % 2 == max_player and run_minimax and game.status == 'playing':
            val, best_move = minimax(game, depth, max_player)
            print("EVALUATION:", val)
            print("BEST MOVE:", best_move)
            w = game.check_winner()
            if w is not None:
                print("Kraj igre, pobjednik:", w)
                game.status = 'game over'
            else:
                if best_move:
                    piece, to_loc, jumped = best_move
                    real_piece = game.piece_at(piece.row, piece.col)
                    real_jumped = game.piece_at(jumped.row, jumped.col) if jumped else None
                    game.play(real_piece, to_loc, real_jumped, auto=True)

                print("Figure:", game.tokens, "Kraljevi:", game.kings)
        else:
            # Unos korisnika
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    done = True
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_pos = pygame.mouse.get_pos()
                    # Ako se pritisne reset button
                    if reset_button_rect.collidepoint(mouse_pos):
                        # Reset trenutne logike
                        game.reset_current_turn()
                    else:
                        # Pass drugi klik
                        game.evaluate_click(mouse_pos)

        # Prikaz svega
        screen.fill(BLACK)
        game.draw(screen)
        pygame.draw.rect(screen, BORDER, (0, HEIGHT, WIDTH, FOOTER_HEIGHT))
        reset_button_rect = draw_reset_button(screen)
        pygame.display.flip()
        sat.tick(60)

    pygame.quit()


if __name__ == "__main__":
    main()

#pygame.font.get_fonts()

EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE8EAEC00>, (3, 0), None)
Figure: [12, 12] Kraljevi: [0, 0]
EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE4DED760>, (2, 1), None)
Figure: [12, 12] Kraljevi: [0, 0]
EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE55BCA10>, (1, 0), None)
Figure: [12, 12] Kraljevi: [0, 0]
EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE4DED760>, (3, 2), None)
Figure: [12, 12] Kraljevi: [0, 0]
EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE55BCA10>, (2, 1), None)
Figure: [12, 12] Kraljevi: [0, 0]
Invalid move.
EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE4DED760>, (4, 1), None)
Figure: [12, 12] Kraljevi: [0, 0]
EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE8EAED80>, (3, 2), None)
Figure: [12, 12] Kraljevi: [0, 0]
EVALUATION: 0.0
BEST MOVE: (<__main__.Figura object at 0x0000025AE8EAD7C0>, (2, 3), None)
Figure: [12, 12] Kraljevi: [